In [1]:
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam, SGD, Adadelta
from sklearn.metrics import classification_report, confusion_matrix
from random import  shuffle
import matplotlib.pyplot as plt
from keras import initializers
from keras.initializers import he_normal
from batch_generator import *
from keras import applications as apps
from PIL import  Image
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path_train = os.path.join('drive/My Drive/Preprocessed_data','Train')
path_test = os.path.join('drive/My Drive/Preprocessed_data','Test')
path_val = os.path.join('drive/My Drive/Preprocessed_data','Val')
train_info, train_data, separated_classes_train = get_train_data_names(path_train,disp = True)
val_info, val_data, separated_classes_val = get_val_data_names(path_val,disp = True)
test_info, test_data = get_test_data(path_test, disp = True)

,classes,labels,Initial train count,Augmented train count
0,Benign,0,57.0,15960
1,InSitu,1,52.0,14560
2,Invasive,2,51.0,14280
3,Normal,3,45.0,12600


,classes,labels,Initial val count,Augmented val count
0,Benign,0,12.0,3360
1,InSitu,1,11.0,3080
2,Invasive,2,11.0,3080
3,Normal,3,10.0,2800


/content/batch_generator.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data['Name'] = names


,classes,labels,Initial test count,Augmented test count
0,Benign,0,9.0,864
1,InSitu,1,9.0,864
2,Invasive,2,9.0,864
3,Normal,3,9.0,864


In [0]:
def build_model(image_shape:tuple = (512,512,3) ,c: int = 4, disp: bool = False):
    
    """
    Builds cnn model
    Arguments:
        image_shape: tuple of integers, example (height, width, nb_channels)
        c: number of output in a final layer (number of classes)
        disp: possible values {True, False}, default value is False. If set to True
        the function displays model summary
    
    Returns:
        cancer_cnn: cnn model
    
    """
    
    x_input = Input(image_shape)
    
    x = Conv2D(16, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x_input)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3)(x)

    x = Conv2D(32, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)
    
        
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3 )(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3 )(x)
    
    
    x = Flatten()(x)
    
    x = Dropout(rate=0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=128, activation='relu')(x)
    y = Dense(c, activation='softmax')(x)
    
    
    cancer_cnn = Model(inputs = x_input, outputs = y, name = 'cancer_cnn')
    
    if disp == True:
        display(cancer_cnn.summary())
    return cancer_cnn
    
    

In [0]:
def model_3(image_shape:tuple = (512,512,3) ,c: int = 4):
    
    """
    Builds cnn model
    Arguments:
        image_shape: tuple of integers, example (height, width, nb_channels)
        c: number of output in a final layer (number of classes)
        disp: possible values {True, False}, default value is False. If set to True
        the function displays model summary
    
    Returns:
        cancer_cnn: cnn model
    
    """
    
    x_input = Input(image_shape)
    
    x = Conv2D(16, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x_input)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3)(x)

    x = Conv2D(32, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)
    
        
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3 )(x)
    
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3 )(x)
    
    
    x = Flatten()(x)
    
    x = Dropout(rate=0.2)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=128, activation='relu')(x)
    y = Dense(c, activation='softmax')(x)
    
    
    cancer_cnn = Model(inputs = x_input, outputs = y, name = 'cancer_cnn')
    
    
    display(cancer_cnn.summary())
    return cancer_cnn
    
    

In [0]:
def model_4(image_shape:tuple = (512,512,3) ,c: int = 4):
    
    """
    Builds cnn model
    Arguments:
        image_shape: tuple of integers, example (height, width, nb_channels)
        c: number of output in a final layer (number of classes)
        disp: possible values {True, False}, default value is False. If set to True
        the function displays model summary
    
    Returns:
        cancer_cnn: cnn model
    
    """
    
    x_input = Input(image_shape)
    
    x = Conv2D(16, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x_input)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3)(x)

    x = Conv2D(32, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size= 2, strides = 2)(x)
    
   
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size = 3, strides = 3 )(x)
  
    
    """
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer= initializers.he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = GlobalMaxPooling2D()(x)
    
    """
    
    x = Flatten()(x)
    
    x = Dropout(rate=0.3)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=128, activation='relu')(x)
    y = Dense(c, activation='softmax')(x)
    
    
    cancer_cnn = Model(inputs = x_input, outputs = y, name = 'cancer_cnn')
    
    
    display(cancer_cnn.summary())
    return cancer_cnn
    
    

In [0]:
def model_2(image_shape = (512,512,3)):
    
    x_input = Input(image_shape)
    
    #blok1
    x  = Conv2D(16, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x_input)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(16, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(16, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
        
        
    #block 2
    
    
    x  = Conv2D(32, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(32, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(32, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    #block3
    
    x  = Conv2D(64, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(64, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(64, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    
    #block4
    
    x  = Conv2D(128, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(128, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(128, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    
    #block5
    
    x  = Conv2D(256, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(256, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(256, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation = 'relu')(x)
    y = Dense(4, activation='softmax')(x)
    
    cancer_cnn = Model(inputs = x_input, outputs =y, name = 'cancer_cnn')
    
    #cancer_cnn.summary()
    
    return cancer_cnn

In [0]:
def model_6(image_shape = (512,512,3)):
    
    x_input = Input(image_shape)
    
    
    
    #blok1
    x  = Conv2D(16, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x_input)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(16, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    
    """
    x = Activation('relu')(x)
    x  = Conv2D(16, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
        
    """ 
    #block 2
    
    
    x  = Conv2D(32, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(32, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    """
    x  = Conv2D(32, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    """
    
    #block3
    
    x  = Conv2D(64, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(64, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    """
    x  = Conv2D(64, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    """
    
    #block4
    
    x  = Conv2D(128, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(128, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    """
    x  = Conv2D(128, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    """
    
    #block5
    
    x  = Conv2D(256, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x  = Conv2D(256, (3, 3), strides=(1, 1), padding = 'valid',kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    """
    x  = Conv2D(256, (2, 2), strides=(2, 2),kernel_initializer= he_normal())(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    """
    
    
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation = 'relu')(x)
    y = Dense(4, activation='softmax')(x)
    
    cancer_cnn = Model(inputs = x_input, outputs =y, name = 'cancer_cnn')
    
    #cancer_cnn.summary()
    
    return cancer_cnn

In [0]:
def model_transfer():
  
  
  model = apps.VGG16(weights = 'imagenet',include_top=False, input_shape=(512, 512, 3))
  
  
  
  x = model.get_layer('block5_pool').output
  # add new classifier layers
 
  x = Flatten()(x)
  x = Dropout(0.3)(x)
  x = Dense(8, activation='relu')(x)
  output = Dense(4, activation='softmax')(x)
  # define new model
  model = Model(inputs=model.inputs, outputs=output)
  # summarize
  
  
  
  for layer in model.layers[:15]:
    layer.trainable = False
  
  #model.summary()
  
  return model

  

In [0]:
class PatchModel():
    
    def __init__(self,img_shape = (512,512,3), c = 4, disp = False):
        
        self.model = build_model(image_shape = img_shape, c = c, disp = disp)
        
        
        #self.model = model_2()  
        #self.model = model_4()
        #self.model = model_3()
        #self.model = model_transfer()
       
        
        
    def load(self, filename: 'path'):
        self.model = load_model(filename)
        #return self.model

    def save(self: 'cnn model', filename: 'path'):
        self.model.save(filename)
    
    def compile_model(self: 'cnn model',lr = 0.00075):
      
      
        #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        #adam = Adam(lr=lr)
        
        adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
        
        a = self.model.compile(optimizer=adadelta,
                               loss='categorical_crossentropy', metrics=['accuracy'])
        
        return a
    
    def fit(self, train_samples, val_samples, batch_size = 35, epochs = 100):
        
        """
        Fits model using BatchGenerator generator using train and validation sets
        
        Arguments:
            samples: DataFrame with names of train data files and labels given as (rowvise):
                    ['file_1.tif', label_1 encoded]
                    ['file_2'.tiff, label_2 encoded]
                    .
                    .
                    .
                    ['file_n.tif', label_n encoded]
        
            batch_size: size of a single batch
            epochs: number of epochs
            
        Returns: 
            History object
        
        """
        #filepath = './checkpoints/chekpoint' + "-final-{epoch:02d}-{val_acc:.2f}.hdf5"
        #callbacks = [EarlyStopping(monitor='acc', min_delta=0, patience=3, mode='auto'),
        #            ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)]
        
        
        callbacks = [EarlyStopping(monitor='acc', min_delta=0, patience=3, mode='auto')]
       
        
        steps_per_epoch = np.ceil(train_samples.shape[0]/batch_size)
        
        validation_steps = np.ceil(val_samples.shape[0]/batch_size)
     
            
        train_generator = BatchGenerator(train_samples, batch_size = batch_size).generate()
        val_generator = BatchGenerator(val_samples, batch_size = batch_size).generate()
    
        return self.model.fit_generator(train_generator, validation_data = val_generator,
                                        callbacks=callbacks,
                                        steps_per_epoch = steps_per_epoch,
                                        validation_steps = validation_steps,
                                        epochs = epochs
                                       )
    
    def evaluate(self, samples: pd.DataFrame, batch_size:int = 35, steps = 1):
        
        """
        Evaluates model.
        
        Arguments:
            samples: DataFrame with names of train data files and labels given as (rowvise):
                    ['file_1.tif', label_1 encoded]
                    ['file_2'.tiff, label_2 encoded]
                    .
                    .
                    .
                    ['file_n.tif', label_n encoded]
            batch_size: size of a single batch
            
        Returns:
            History objest
        
        """        
        steps = np.ceil(len(samples)/batch_size)
        test_data = BatchGenerator(samples,batch_size = batch_size).generate()
        
        return self.model.evaluate_generator(generator = test_data, steps = steps)
    
    
    def batch_predict(self, samples, steps = 1, batch_size = 32):
        
        generator = BatchGenerator(samples).generate()
        
        steps = np.ceil(len(samples)/batch_size)
        
        return self.model.predict_generator(generator, steps=steps)
    
    
   
    
   
    
    def plot_accuracy(history: 'History object'):
        
        """
        Plots accuracy and  validation accuracy regarding to epochs
        Arguments:
            history: Model's history
        
        """
        
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()

    
    def plot_loss(history: 'History object'):
        
        """
        Plots loss and validation loss regarding to epochs
        Arguments:
            hisroty: Model's history
        
        """
        
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()
        
    def get_features(self, samples_train: pd.DataFrame, samples_test:pd.DataFrame,
                     layer:str = 'sloj', disp:bool = False):
    
        model_features = Model(inputs = self.model.input, outputs = self.model.get_layer(layer).output)
    
        data_train = BatchGenerator(samples_train, batch_size= len(samples_train)).generate()
        data_test = BatchGenerator(samples_test).generate()
    
        feat_train = model_features.predict_generator(data_train, steps =  1)
        feat_test = model_features.predict_generator(data_test, steps = 1)
    
        if disp == True:
            print(f"Features train shape: {feat_train.shape}")
            print(f"Features test shape: {feat_test.shape}")

        
        return {'features_train' : feat_train, 'features_test' : feat_test} 
    

In [0]:
 def img_predict(model, paths:'list of image paths'):
        
        voting_preds = []
        
        for img_path in paths:
            
        
            img = np.array(Image.open(img_path))
            patches,_,_ = Data_augmentation.py_patch(img)

            predictions_proba =  model.predict(np.array(patches))

            
            predictions_class = list(np.argmax(predictions_proba, axis = 1))

   
            #majority voting
            majority_prediction = max(predictions_class, key=predictions_class.count)
    
    
            #maximum propability voting
            patch_index = np.argmax(np.max(predictions_proba, axis = 1))
            max_prob_prediction = np.argmax(predictions_proba[patch_index], axis = 0)
        
            #sum of probabilities voting
            sum_prob_prediction = np.argmax(np.sum(predictions_proba, axis = 0 ))
        
        
            voting_preds.append( {'Majority voting': majority_prediction,
                                  'Maximum probability': max_prob_prediction,
                                  'Sum of probabilities': sum_prob_prediction})
    

            #print(voting_preds)
        
        return voting_preds

In [0]:
cnn_1 = PatchModel()
cnn_1.compile_model()
cnn_1.fit(train_data, val_data, epochs = 5)

In [0]:
cnn_1.save('drive/My Drive/model_6_.h5')

In [0]:
cnn_1.evaluate(test_data)